In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import SRN2VecModel
import json

In [2]:
city = "sf"

In [3]:
network = RoadNetwork()
network.load(f"../../osm_data/{city}")

In [4]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network)

In [5]:
model.generate_data(n_shortest_paths=100, number_negative=3, window_size=900, save_batch_size=100, city=city)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
%lprun -f model.extract_pairs model.generate_train_pairs(network, paths, 900, 7)

In [3]:
with open(f"srn2vec-traindata-{city}.json", "r") as fp:
    a = json.load(fp)

In [7]:
"""
Build and train model
"""
network = RoadNetwork()
network.load(f"../../osm_data/{city}")
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network, remove_highway_label=True)
model.load_dataset(f"./srn2vec-traindata-{city}.json")

In [8]:
model.train(epochs=5, batch_size=256)

Epoch: 0, Iteration: 0, sample_loss: 0.7457817792892456, Avg. Loss: 0.7457817792892456
Epoch: 0, Iteration: 1000, sample_loss: 0.52885502576828, Avg. Loss: 0.5871139772228904
Epoch: 0, Iteration: 2000, sample_loss: 0.41753605008125305, Avg. Loss: 0.5420324301820942
Epoch: 0, Iteration: 3000, sample_loss: 0.44189512729644775, Avg. Loss: 0.5227178169246198
Epoch: 0, Iteration: 4000, sample_loss: 0.4610782861709595, Avg. Loss: 0.5124071226347032
Epoch: 0, Iteration: 5000, sample_loss: 0.4362286627292633, Avg. Loss: 0.5063494576666027
Epoch: 0, Iteration: 6000, sample_loss: 0.4377515912055969, Avg. Loss: 0.5021882874183785
Epoch: 0, Iteration: 7000, sample_loss: 0.48308855295181274, Avg. Loss: 0.49940201709090326
Epoch: 0, Iteration: 8000, sample_loss: 0.38562193512916565, Avg. Loss: 0.4971428117578051
Epoch: 0, Iteration: 9000, sample_loss: 0.45629215240478516, Avg. Loss: 0.49559881924298216
Epoch: 0, Iteration: 10000, sample_loss: 0.4468163549900055, Avg. Loss: 0.49399206595961515
Epoch:

In [9]:
model.save_model(path="../model_states/srn2vec/")